# Novel Information Visualization

# Anastasiia Deviataieva, Student Number: 24100519

# Data preprocessing

Import required packages.

In [4]:
import pandas as pd

This code looks at Formula 1 race data to find out how often drivers win depending on where they start on the grid. It starts by loading race results, driver status (whether they finished, crashed, etc.), and team names. Then it removes any drivers who didn’t start the race and keeps only the real winners—those who finished first and completed the race. After that, it checks which team won the most from each starting position. It also counts how many times drivers started from each spot and how many times they won. Then it calculates the win rate (how often that position leads to a win) and saves all the results in a `histogram_summary.json` file.

In [8]:
# WIN RATE BY STARTING POSITION
# Base URL for raw CSV files in the GitHub repository
BASE = "https://raw.githubusercontent.com/an-mistral/Developing_a_Novel_InfoVis/refs/heads/main/Formula1_data"
# BASE = BASE.rstrip("/")
URL_RESULTS      = f"{BASE}/results.csv"
URL_STATUS       = f"{BASE}/status.csv"
URL_CONSTRUCTORS = f"{BASE}/constructors.csv"

# Load source data
results = pd.read_csv(URL_RESULTS,           encoding="latin-1")
status = pd.read_csv(URL_STATUS,             encoding="latin-1")
constructors = pd.read_csv(URL_CONSTRUCTORS, encoding="latin-1")

# Merge status descriptions into race results
results = results.merge(status, on="statusId", how="left")

# Filter out drivers who did not start (grid = -1 or 0)
results = results[results["grid"] >= 1]

# Select race winners who finished normally (positionOrder == 1 and not DNF or "Lapped")
winners = results[
    (results["positionOrder"] == 1) &
    (~results["status"].str.contains("Lap", na=False))
][["raceId", "driverId", "constructorId", "grid"]]

# Merge in constructor names
winners = winners.merge(constructors[["constructorId", "name"]], on="constructorId", how="left")
winners = winners.rename(columns={
    "grid": "start_position",
    "name": "top_constructor"})

# Count wins by starting position and most frequent winning constructor for each
grouped_wins = winners.groupby("start_position").agg(
    wins=('raceId', 'count'),
    top_constructor=('top_constructor', lambda x: x.value_counts().idxmax())
).reset_index()

# Count total starts per starting position
starts = results.groupby("grid").size().reset_index(name="starts")
starts = starts.rename(columns={"grid": "start_position"})

# Merge wins and starts, then calculate win rate
summary = pd.merge(grouped_wins, starts, on="start_position", how="left")
summary["win_rate"] = (summary["wins"] / summary["starts"]) * 100

# Clean up and sort by win rate
summary = summary.dropna()
summary["start_position"] = summary["start_position"].astype(int)
summary = summary.sort_values("win_rate", ascending=False)

# Save result to JSON
summary.drop(columns=["starts"]).to_json("histogram_summary.json", orient="records")

This block takes data on races, constructor teams, and their points, combines them, calculates how many points each team scored in each year, and saves this summary table to a file named `constructor_points_by_year.csv`.

In [10]:
#CONSTRUCTOR POINTS BY SEASON HEATMAP
# Base URL for raw CSV files in the GitHub repository
BASE = "https://raw.githubusercontent.com/an-mistral/Developing_a_Novel_InfoVis/refs/heads/main/Formula1_data"
URL_RACES        = f"{BASE}/races.csv"
URL_CONSTRUCTORS = f"{BASE}/constructors.csv"
URL_STAND        = f"{BASE}/constructorStandings.csv"

# Load source data (only necessary columns)
races = pd.read_csv(URL_RACES,                 encoding="latin-1", usecols=["raceId", "year"])
constructors = pd.read_csv(URL_CONSTRUCTORS,   encoding="latin-1", usecols=["constructorId", "name"])
constructor_standings = pd.read_csv(URL_STAND, encoding="latin-1", usecols=["raceId", "constructorId", "points"])

# Join year + constructor name onto standings and aggregate season points
merged = constructor_standings.merge(races, on="raceId", how="left")
merged = merged.merge(constructors, on="constructorId", how="left")

grouped = (merged.groupby(["year", "name"], as_index=False)["points"].sum())
grouped.to_csv("constructor_points_by_year.csv", index=False)

This script prepares a season‐level summary that pairs each driver’s average lap pace with their total championship points. It pulls raw lap‐time records, race metadata, driver standings, and driver info, then converts lap times from milliseconds into seconds and tags each lap with its race year. Next, it groups by driver and year to compute the mean lap time across all events in a season. To get final points, it finds the last race of each year and selects the corresponding driver standings entry, renaming that cumulative total as seasonPoints. After merging average lap times and season points, the code attaches full driver names (forename + surname) for readability and writes out a clean CSV—`avgLapTime_vs_seasonPoints.csv`—with columns driver, year, avgLapTimeYear, and seasonPoints ready for visualization.

In [12]:
# Base URL for raw CSV files in the GitHub repository
BASE = "https://raw.githubusercontent.com/an-mistral/Developing_a_Novel_InfoVis/refs/heads/main/Formula1_data"
URL_LAPS       = f"{BASE}/lapTimes.csv"
URL_RACES      = f"{BASE}/races.csv"
URL_DRIVER_ST  = f"{BASE}/driverStandings.csv"
URL_DRIVERS    = f"{BASE}/drivers.csv"

# Load lap times, race metadata, driver standings, and driver info
lap_times       = pd.read_csv(URL_LAPS,       encoding="latin-1")
races           = pd.read_csv(URL_RACES,      encoding="latin-1")[["raceId", "year"]]
driver_standings= pd.read_csv(URL_DRIVER_ST,  encoding="latin-1")
drivers         = pd.read_csv(URL_DRIVERS,    encoding="latin-1")[["driverId", "forename", "surname"]]

# Convert lap times from milliseconds to seconds
lap_times["lapTimeSec"] = lap_times["milliseconds"] / 1000

# Attach the year of each race to its lap times
lap_times = lap_times.merge(races, on="raceId", how="left")

# Compute each driver’s average lap time for the entire year
avg_lap_year = (lap_times
                .groupby(["driverId", "year"], as_index=False)["lapTimeSec"]
                .mean()
                .rename(columns={"lapTimeSec": "avgLapTimeYear"}))

# Identify the final race of each season to get season-ending points
last_race = races.groupby("year", as_index=False)["raceId"].max()

# Filter driver standings to only those final races
final_stand = driver_standings.merge(
    last_race, on="raceId", how="inner"
)[["driverId", "year", "points"]].rename(columns={"points": "seasonPoints"})

# Merge average lap times with season-ending points
df = avg_lap_year.merge(final_stand, on=["driverId", "year"], how="inner")

# Add full driver names for readability
drivers["driver"] = drivers["forename"] + " " + drivers["surname"]
df = df.merge(drivers[["driverId", "driver"]], on="driverId", how="left")

# Select and sort final columns, then save to CSV
out = df[["driver", "year", "avgLapTimeYear", "seasonPoints"]]
out = out.sort_values(["year", "seasonPoints"], ascending=[True, False])
out.to_csv("avgLapTime_vs_seasonPoints.csv", index=False)

print(f"avgLapTime_vs_seasonPoints.csv saved | rows: {len(out)}")

avgLapTime_vs_seasonPoints.csv saved | rows: 536


This script downloads Formula 1 race results and driver data, filters out the race winners, merges in each driver’s nationality, and then aggregates the total number of wins by country. The final tally is saved to `wins_by_country.csv` for further analysis or visualization.

In [14]:
# Base URL for raw CSV files in the GitHub repository
BASE = "https://raw.githubusercontent.com/an-mistral/Developing_a_Novel_InfoVis/refs/heads/main/Formula1_data"
URL_RESULTS = f"{BASE}/results.csv"
URL_DRIVERS = f"{BASE}/drivers.csv"

# Load race results and driver information
results = pd.read_csv(URL_RESULTS, encoding="latin-1")
drivers = pd.read_csv(URL_DRIVERS, encoding="latin-1")

# Select only race winners (drivers who finished in position 1)
winners = results[results["positionOrder"] == 1][["driverId"]]

# Attach each winner’s nationality
winners = winners.merge(drivers[["driverId", "nationality"]], on="driverId", how="left")

# Count total wins per country
wins_by_country = (winners.groupby("nationality")
                   .size()
                   .reset_index(name="wins")
                   .rename(columns={"nationality": "country"})
                   .sort_values("wins", ascending=False))

# Write the country-level win totals to CSV
wins_by_country.to_csv("wins_by_country.csv", index=False)
print(f"wins_by_country.csv saved  | rows: {len(wins_by_country)}")

wins_by_country.csv saved  | rows: 22


This script produces a CSV of annual Formula 1 constructor champions enriched with decade labels, last‐digit positioning, and logo URLs. It loads season standings, finds each season’s final race to determine the champion, merges in constructor names, and ensures a single champion per year by sorting on final position and points. It then computes decade (e.g., “196X”) and digit (0–9), maps each champion to a predefined logo URL (including alternate name synonyms), and alerts if any logos are missing. The final table—containing year, decade, digit, name, and logo—is saved as `champions_logos.csv` for use in visual dashboards.

In [6]:
#CONSTRUCTOR CHAMPIONS BY SEASON
# Base URL for raw CSVs in the GitHub repo
BASE = "https://raw.githubusercontent.com/an-mistral/Developing_a_Novel_InfoVis/refs/heads/main/Formula1_data"
BASE_2 = "https://raw.githubusercontent.com/an-mistral/Developing_a_Novel_InfoVis/refs/heads/main"
URL_STAND = f"{BASE}/constructorStandings.csv"
URL_CONS  = f"{BASE}/constructors.csv"
URL_RACES = f"{BASE}/races.csv"

# Load constructor standings, constructor names, and race years
stand = pd.read_csv(URL_STAND, encoding="latin-1")
cons  = pd.read_csv(URL_CONS,  encoding="latin-1")[["constructorId", "name"]]
races = pd.read_csv(URL_RACES, encoding="latin-1")[["raceId", "year"]]

# Find the final race ID of each season
last_race = races.groupby("year")["raceId"].max().reset_index()

# Merge standings with last_race to attach 'year', then merge in constructor names
merged = (stand
          .merge(last_race, on="raceId")        # add 'year' for the season finale
          .merge(cons, on="constructorId"))      # add constructor 'name'

# Ensure 'year' column is present
assert "year" in merged.columns

# Select a single champion per year: sort by season, position, then points descending
champ = (merged
         .sort_values(["year", "position", "points"], ascending=[True, True, False])
         .groupby("year", as_index=False)
         .first()[["year", "constructorId", "name"]])

# Compute decade label (e.g., "196X") and last digit of year
champ["decade"] = champ["year"].astype(str).str[:-1] + "X"
champ["digit"]  = champ["year"] % 10

# Dictionary of logo URLs keyed by constructor name
logos = {
    "Vanwall":  f"{BASE_2}/images/Vanwall.png",
    "Cooper":   f"{BASE_2}/images/Cooper.png",
    "Ferrari":  f"{BASE_2}/images/ferrari.png",
    "BRM":      f"{BASE_2}/images/BRM.png",
    "Lotus":    f"{BASE_2}/images/lotus.png",
    "Brabham":  f"{BASE_2}/images/Repco.png",
    "Matra":    f"{BASE_2}/images/Matra.png",
    "Tyrrell":  f"{BASE_2}/images/tyrrel.png",
    "McLaren":  f"{BASE_2}/images/mclaren.png",
    "Williams": f"{BASE_2}/images/williams.png",
    "Benetton": f"{BASE_2}/images/benetton.png",
    "Renault":  f"{BASE_2}/images/renault.png",
    "Brawn":    f"{BASE_2}/images/Brawn.png",
    "Brawn GP": f"{BASE_2}/images/Brawn.png",
    "Red Bull": f"{BASE_2}/images/oracle_red_bull.png",
    "Red Bull Racing": f"{BASE_2}/images/oracle_red_bull.png",
    "Mercedes": f"{BASE_2}/images/mercedes-benz.png",
    "Mercedes GP": f"{BASE_2}/images/mercedes-benz.png"}

# Add synonym keys for alternate constructor names
logos.update({
    "Cooper-Climax": logos["Cooper"],
    "Lotus-Climax":  logos["Lotus"],
    "Brabham-Repco": logos["Brabham"],
    "Lotus-Ford":    logos["Lotus"],
    "Matra-Ford":    logos["Matra"],
    "Team Lotus":    logos["Lotus"]})

# Map each champion to its logo URL
champ["logo"] = champ["name"].map(logos)

# Warn if any constructors lack a logo entry
missing = champ[champ["logo"].isna()]["name"].unique()
if len(missing):
    print("No logo found for:", ", ".join(missing))

# Output final CSV with columns: year, decade, digit, constructor name, and logo URL
champ[["year", "decade", "digit", "name", "logo"]] \
    .to_csv("champions_logos.csv", index=False)

print(f"champions_logos.csv generated with {len(champ)} rows")

champions_logos.csv generated with 60 rows


This script pulls race results and race metadata, limits the data to the 2010–2017 seasons, and then computes two key metrics for each Grand Prix: the fastest lap set by the race winner and the outright fastest lap of the event. It converts lap‐time strings into numeric seconds, merges the winner and overall fastest‐lap records with the race list, and builds a “Grand Prix, Year” label. Finally, it outputs a CSV (`gp_lap_compare_2010_2017.csv`) containing the race identifier, year, winner’s lap time in seconds, and overall fastest lap time in seconds, ready for further analysis or visualization.

In [18]:
# FASTEST LAP VS WINNER'S BEST LAP
# Base URL for raw CSV files in the GitHub repository
BASE = "https://raw.githubusercontent.com/an-mistral/Developing_a_Novel_InfoVis/refs/heads/main/Formula1_data"
URL_RESULTS = f"{BASE}/results.csv"
URL_RACES   = f"{BASE}/races.csv"

# Load only the necessary columns from results and races
results = pd.read_csv(URL_RESULTS, encoding="latin-1", usecols=["raceId", "positionOrder", "fastestLapTime"])
races = pd.read_csv(URL_RACES, encoding="latin-1", usecols=["raceId", "year", "name"])

# Helper function: convert a "M:SS.mmm" string into total seconds
def lap_to_sec(txt):
    if pd.isna(txt) or txt in ("\\N", ""):
        return None
    m, s = txt.split(":")
    return int(m) * 60 + float(s)

# Filter to seasons between 2010 and 2017 inclusive
period = races[(races["year"] >= 2010) & (races["year"] <= 2017)]

# Extract each race winner’s fastest lap time
winner = (results[results["positionOrder"] == 1]
          .rename(columns={"fastestLapTime": "winnerLap"})
          [["raceId", "winnerLap"]])

# Identify the overall fastest lap in each race
flap = (results[results["fastestLapTime"].notna()]
        .assign(sec=lambda df: df["fastestLapTime"].map(lap_to_sec))
        .sort_values(["raceId", "sec"])
        .groupby("raceId", as_index=False)
        .first()[["raceId", "fastestLapTime"]]
        .rename(columns={"fastestLapTime": "fastestLap"}))

# Merge the period races with winner and fastest-lap data
df = (period
      .merge(winner, on="raceId", how="left")
      .merge(flap,    on="raceId", how="left"))

# Convert lap times to seconds and build a human-readable GP label
df["winnerLapSec"]  = df["winnerLap"].map(lap_to_sec)
df["fastestLapSec"] = df["fastestLap"].map(lap_to_sec)
df["gp"] = df["name"] + ", " + df["year"].astype(str)

# Select and sort the final output, then save to CSV
out = df[["gp", "year", "fastestLapSec", "winnerLapSec"]].sort_values("year")
# Shorten GP labels: replace "Grand Prix" with "GP" for more compact chart text.
out["gp"] = out["gp"].str.replace("Grand Prix", "GP", regex=False)

out.to_csv("gp_lap_compare_2010_2017.csv", index=False)
print(f"gp_lap_compare_2010_2017.csv saved | rows: {len(out)}")

gp_lap_compare_2010_2017.csv saved | rows: 156
